In [1]:
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-10.parquet

--2025-03-11 22:35:57--  https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-10.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 18.155.128.222, 18.155.128.46, 18.155.128.187, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|18.155.128.222|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 64346071 (61M) [binary/octet-stream]
Saving to: ‘yellow_tripdata_2024-10.parquet’

yellow_tripdata_202 100%[===================>]  61.36M   155MB/s    in 0.4s    

2025-03-11 22:35:58 (155 MB/s) - ‘yellow_tripdata_2024-10.parquet’ saved [64346071/64346071]



In [10]:
!ls

01_test.ipynb		  re_yellow
02_yellow_2024_oct.ipynb  yellow_tripdata_2024-10.parquet


In [1]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('homework') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/11 22:47:32 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df = spark.read \
    .option("header", "true") \
    .parquet('yellow_tripdata_2024-10.parquet')

In [4]:
df.repartition(4).write.parquet("re_yellow", mode="overwrite")

In [6]:
!ls -lh re_yellow

total 90M
-rw-r--r-- 1 zotya zotya   0 Mar 11 22:47 _SUCCESS
-rw-r--r-- 1 zotya zotya 23M Mar 11 22:47 part-00000-f67d0f0e-4550-4a36-8e90-c9b27db22a88-c000.snappy.parquet
-rw-r--r-- 1 zotya zotya 23M Mar 11 22:47 part-00001-f67d0f0e-4550-4a36-8e90-c9b27db22a88-c000.snappy.parquet
-rw-r--r-- 1 zotya zotya 23M Mar 11 22:47 part-00002-f67d0f0e-4550-4a36-8e90-c9b27db22a88-c000.snappy.parquet
-rw-r--r-- 1 zotya zotya 23M Mar 11 22:47 part-00003-f67d0f0e-4550-4a36-8e90-c9b27db22a88-c000.snappy.parquet


In [7]:
df.createOrReplaceTempView('yellow_10_24')

In [8]:
df.columns

['VendorID',
 'tpep_pickup_datetime',
 'tpep_dropoff_datetime',
 'passenger_count',
 'trip_distance',
 'RatecodeID',
 'store_and_fwd_flag',
 'PULocationID',
 'DOLocationID',
 'payment_type',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'improvement_surcharge',
 'total_amount',
 'congestion_surcharge',
 'Airport_fee']

In [42]:
spark.sql("""
SELECT 
    EXTRACT(MONTH FROM tpep_pickup_datetime) AS Month,
    EXTRACT(DAY FROM tpep_pickup_datetime) AS Day,
    COUNT(1) AS trips
FROM yellow_10_24
WHERE EXTRACT(DAY FROM tpep_pickup_datetime) = 15
GROUP BY Month, Day;
""").show()

+-----+---+------+
|Month|Day| trips|
+-----+---+------+
|   10| 15|128893|
+-----+---+------+



In [46]:
spark.sql("""
SELECT tpep_pickup_datetime, tpep_dropoff_datetime,
    DATEDIFF(hour, tpep_pickup_datetime, tpep_dropoff_datetime) AS trip_time_hh
FROM yellow_10_24
ORDER BY trip_time_hh DESC;
""").show()

+--------------------+---------------------+------------+
|tpep_pickup_datetime|tpep_dropoff_datetime|trip_time_hh|
+--------------------+---------------------+------------+
| 2024-10-16 13:03:49|  2024-10-23 07:40:53|         162|
| 2024-10-03 18:47:25|  2024-10-09 18:06:55|         143|
| 2024-10-22 16:00:55|  2024-10-28 09:46:33|         137|
| 2024-10-18 09:53:32|  2024-10-23 04:43:37|         114|
| 2024-10-20 13:30:52|  2024-10-24 06:57:38|          89|
| 2024-10-21 00:36:24|  2024-10-24 18:30:18|          89|
| 2024-10-22 16:04:52|  2024-10-25 14:22:49|          70|
| 2024-10-12 19:32:51|  2024-10-15 15:07:15|          67|
| 2024-10-17 17:58:18|  2024-10-20 12:02:18|          66|
| 2024-10-21 14:28:21|  2024-10-23 12:53:42|          46|
| 2024-10-20 18:58:28|  2024-10-22 13:17:00|          42|
| 2024-10-24 20:29:58|  2024-10-26 10:58:25|          38|
| 2024-10-23 23:52:02|  2024-10-25 09:49:06|          33|
| 2024-10-02 10:36:50|  2024-10-03 12:54:45|          26|
| 2024-10-14 2

In [47]:
!wget https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv

--2025-03-11 23:26:58--  https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 18.155.128.6, 18.155.128.46, 18.155.128.187, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|18.155.128.6|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12331 (12K) [text/csv]
Saving to: ‘taxi_zone_lookup.csv’

taxi_zone_lookup.cs 100%[===================>]  12.04K  --.-KB/s    in 0s      

2025-03-11 23:26:58 (293 MB/s) - ‘taxi_zone_lookup.csv’ saved [12331/12331]



In [48]:
df_zones = spark.read \
    .option("header", "true") \
    .csv('taxi_zone_lookup.csv')

In [49]:
df_zones.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [54]:
df_join = df.join(df_zones, df.PULocationID == df_zones.LocationID)

In [55]:
df_join.createOrReplaceTempView('yellow_10_24_zone')

In [58]:
spark.sql("""
SELECT Zone, COUNT(1) AS pickups
FROM yellow_10_24_zone
GROUP BY Zone
ORDER BY pickups ASC;
""").show()

+--------------------+-------+
|                Zone|pickups|
+--------------------+-------+
|Governor's Island...|      1|
|       Rikers Island|      2|
|       Arden Heights|      2|
|         Jamaica Bay|      3|
| Green-Wood Cemetery|      3|
|Charleston/Totten...|      4|
|   Rossville/Woodrow|      4|
|       West Brighton|      4|
|Eltingville/Annad...|      4|
|       Port Richmond|      4|
|         Great Kills|      6|
|        Crotona Park|      6|
|Heartland Village...|      7|
|     Mariners Harbor|      7|
|Saint George/New ...|      9|
|             Oakwood|      9|
|       Broad Channel|     10|
|New Dorp/Midland ...|     10|
|         Westerleigh|     12|
|     Pelham Bay Park|     12|
+--------------------+-------+
only showing top 20 rows

